## UC2: BE_BEER

#### Data Prep

In [54]:
import re
import pandas as pd
from wkf.Columns import Columns

main_dir = '/media/alex/data/alex/work/wkf/nielsen/UC2/BE_BEER/'
train = pd.read_csv(main_dir + 'data/train.csv', sep=',', quotechar='"', encoding='latin-1')
test = pd.read_csv(main_dir + 'data/test.csv', sep=',', quotechar='"', encoding='latin-1')

# normalize headers
train = train.rename(columns=lambda x: re.sub('[^0-9a-zA-Z]', '_', x).lower())
test.columns = map((lambda x: re.sub('[^0-9a-zA-Z]', '_', x).lower()), test.columns)

columns = Columns(country='BE', category='BEER', use_case='uc2', data_set_csv=main_dir + 'data/train.csv')
predictors = columns.get_input_cols()
# remove from predictors column which is not in test set in this case
predictors.pop(predictors.index(u'_be_loc_gamm___m_gamme'))
predictor = 'item_description'
responses = columns.get_output_cols()
response = 'brand_1'
X_train = train[predictor]
y_train = train[response].iloc[0:]
X_test = test[predictor]
y_test = test[response].iloc[0:]


In [55]:
print X_train.shape, y_train.shape, X_test.shape, y_test.shape
# print('X_train first entry:\n\n', X_train.iloc[0])
# print('\n\nX_train shape: ', X_train.shape)
# print X_train.describe()

(5423,) (5423,) (201,) (201,)


# CountVectorizer

In [56]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer().fit(X_train)

In [62]:
vect.get_feature_names()

[u'10',
 u'1000ml',
 u'11',
 u'12',
 u'1200ml',
 u'125',
 u'13',
 u'14',
 u'15',
 u'1500ml',
 u'1664',
 u'1673',
 u'18',
 u'180ml',
 u'1830',
 u'1858',
 u'1886',
 u'1892',
 u'1900',
 u'19500ml',
 u'20000ml',
 u'2000ml',
 u'200ml',
 u'2013',
 u'25',
 u'250ml',
 u'275ml',
 u'277ml',
 u'28',
 u'280ml',
 u'284ml',
 u'285ml',
 u'290ml',
 u'2eur',
 u'30000ml',
 u'3000ml',
 u'300ml',
 u'308ml',
 u'310ml',
 u'314ml',
 u'318ml',
 u'33',
 u'330ml',
 u'349ml',
 u'35',
 u'355ml',
 u'365',
 u'375ml',
 u'4000ml',
 u'400ml',
 u'421',
 u'440ml',
 u'470ml',
 u'472ml',
 u'498ml',
 u'50',
 u'50000ml',
 u'5000ml',
 u'500ml',
 u'540ml',
 u'568ml',
 u'5e',
 u'6000ml',
 u'621',
 u'640ml',
 u'650ml',
 u'66',
 u'660ml',
 u'750ml',
 u'7pk',
 u'8\xe5',
 u'9000ml',
 u'aas',
 u'abbaye',
 u'abbe',
 u'abdij',
 u'abdis',
 u'ac',
 u'achel',
 u'achelse',
 u'achilles',
 u'achouffe',
 u'ada',
 u'adams',
 u'adelardus',
 u'adriaans',
 u'adriaen',
 u'advokaat',
 u'affligem',
 u'agrum',
 u'agrumbocq',
 u'agrume',
 u'ahold',


In [58]:
len(vect.get_feature_names())

1658

In [65]:
# transform the documents in the training data to a document-term matrix
X_train_vectorized = vect.transform(X_train)
X_train_vectorized

<5423x1658 sparse matrix of type '<type 'numpy.int64'>'
	with 71557 stored elements in Compressed Sparse Row format>

Multiclass classufication is not supported (probably) in this case

In [88]:
from sklearn.linear_model import LogisticRegression

# Train the model
print X_train_vectorized.shape, y_train.shape
model = LogisticRegression(multi_class='multinomial', solver='newton-cg')
# model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
model.fit(X_train_vectorized, y_train)

(5423, 1658) (5423,)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)

In [89]:
from sklearn.metrics import roc_auc_score, confusion_matrix

# Predict the transformed test documents
predictions = model.predict(vect.transform(X_test))
# print('AUC: ', roc_auc_score(y_test, predictions))
print('AUC: ', confusion_matrix(y_test, predictions))

# roc = {label: [] for label in y_test.unique()}
# for label in y_test.unique():
#     model = LogisticRegression()
#     model.fit(X_train_vectorized, y_train == label)
#     predictions_proba = model.predict_proba(X_test)
#     roc[label] += roc_auc_score(y_test, predictions_proba[:, 1])

('AUC: ', array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 3, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0]]))


In [90]:
from nltk.classify import NaiveBayesClassifier as nb
from nltk.classify.util import accuracy
cl = nb.train(X_train_vectorized)
cl.classify(X_test)
cl.classify_many(X_test)
accuracy(cl, X_test)
cl.labels()
cl.show_most_informative_features()

ValueError: need more than 1 value to unpack

In [60]:
from sklearn import svm

clfrSVM = svm.SVC(kernel='linear', C=0.1)
clfrSVM.fit(X_train_vectorized, y_train)


SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [61]:
from sklearn import naive_bayes as nb, metrics

clfrNB = nb.MultinomialNB()
clfrNB.fit(X_train_vectorized, y_train_vectorized)
pred = clfrNB.predict(X_test)
metrics.f1_score(y_test, pred, average='micro')


ValueError: bad input shape (5423, 1658)

In [103]:
from sklearn.metrics import roc_auc_score

# Predict the transformed test documents
predictions = model.predict(vect.transform(X_test))

print('AUC: ', roc_auc_score(y_test, predictions))

ValueError: multiclass format is not supported

In [ ]:
# get the feature names as numpy array
feature_names = np.array(vect.get_feature_names())

# Sort the coefficients from the model
sorted_coef_index = model.coef_[0].argsort()

# Find the 10 smallest and 10 largest coefficients
# The 10 largest coefficients are being indexed using [:-11:-1] 
# so the list returned is in order of largest to smallest
print('Smallest Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))

# Tfidf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Fit the TfidfVectorizer to the training data specifiying a minimum document frequency of 5
vect = TfidfVectorizer(min_df=5).fit(X_train)
len(vect.get_feature_names())

In [ ]:
X_train_vectorized = vect.transform(X_train)

model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

predictions = model.predict(vect.transform(X_test))

print('AUC: ', roc_auc_score(y_test, predictions))

In [ ]:
feature_names = np.array(vect.get_feature_names())

sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()

print('Smallest tfidf:\n{}\n'.format(feature_names[sorted_tfidf_index[:10]]))
print('Largest tfidf: \n{}'.format(feature_names[sorted_tfidf_index[:-11:-1]]))

In [ ]:
sorted_coef_index = model.coef_[0].argsort()

print('Smallest Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))

In [ ]:
# These reviews are treated the same by our current model
print(model.predict(vect.transform(['not an issue, phone is working',
                                    'an issue, phone is not working'])))

# n-grams

In [ ]:
# Fit the CountVectorizer to the training data specifiying a minimum 
# document frequency of 5 and extracting 1-grams and 2-grams
vect = CountVectorizer(min_df=5, ngram_range=(1,2)).fit(X_train)

X_train_vectorized = vect.transform(X_train)

len(vect.get_feature_names())

In [ ]:
model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

predictions = model.predict(vect.transform(X_test))

print('AUC: ', roc_auc_score(y_test, predictions))

In [ ]:
feature_names = np.array(vect.get_feature_names())

sorted_coef_index = model.coef_[0].argsort()

print('Smallest Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))

In [ ]:
# These reviews are now correctly identified
print(model.predict(vect.transform(['not an issue, phone is working',
                                    'an issue, phone is not working'])))